In [1]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

def remove_background(image_path, output_path, num_clusters=3):
    """
    Remove background from the image using k-means clustering.

    Parameters:
    - image_path (str): Path to the input image file.
    - output_path (str): Path to save the output image without the background.
    - num_clusters (int): Number of clusters for k-means clustering (default is 3).
    """
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Reshape the image to a 2D array of pixels
    pixels = image_rgb.reshape((-1, 3))
    
    # Apply k-means clustering
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(pixels)
    
    # Get the labels and cluster centers
    labels = kmeans.predict(pixels)
    centers = kmeans.cluster_centers_
    
    # Find the index of the cluster with the lowest luminance (background)
    background_cluster_index = np.argmin(np.linalg.norm(centers, axis=1))
    
    # Assign labels to the background pixels
    background_mask = (labels == background_cluster_index).reshape(image.shape[:2])
    
    # Set background pixels to white
    image[background_mask] = [255, 255, 255]
    
    # Save the output image without the background
    cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

# Example usage:
input_image_path = "D:\Data Science\Dataset\Sugarcane Leaf Disease Dataset Split\train\Healthy\healthy (2).jpeg"
output_image_path = "D:\Data Science\Dataset\Sugarcane Leaf Disease Dataset Split\train\image_without_background.jpg"
remove_background(input_image_path, output_image_path)


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [4]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

# reading the image
img = cv2.imread(r'D:\New_Fourth_Year\projectDesignGit\pestInfestationSeverityLevel\Sugarcane Leaf Disease Dataset Split\train\Yellow\yellow (2).jpeg')

# converting it to rgb
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# flatten the image
img_flat = img.reshape((-1, 3))  # Reshape to a 2D array with pixel rows

# applying k-means clustering
kmeans = KMeans(n_clusters=2, random_state=0).fit(img_flat)  # Adjust n_clusters if needed

# segmenting the image
mask = np.where(kmeans.labels_ == 0, 0, 255).astype(np.uint8)  # Convert the mask to uint8
mask = mask.reshape(img.shape[:2])  # Reshape mask to original image dimensions

img_masked = cv2.bitwise_and(img, img, mask=mask)

# displaying the result of the image
cv2.imshow('Image with Background Removed', img_masked)
cv2.waitKey(0)
cv2.destroyAllWindows()

# To save the image:
cv2.imwrite(r'D:\Data Science\Dataset\Sugarcane Leaf Disease Dataset Split\train\image_without_background.jpg', img_masked)


c:\Users\Jerico\anaconda3\envs\useGPU\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


True

In [5]:
import os
import numpy as np
import cv2
from sklearn.cluster import KMeans

def process_image(image_path, output_root):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image to RGB format

    # Histogram Equalization
    lab_image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    l_channel_eq = cv2.equalizeHist(l_channel)
    lab_image_eq = cv2.merge([l_channel_eq, a_channel, b_channel])
    image_eq = cv2.cvtColor(lab_image_eq, cv2.COLOR_LAB2RGB)

    # Convert the equalized image to LAB
    lab_image_eq = cv2.cvtColor(image_eq, cv2.COLOR_RGB2LAB)

    # Reshape the image
    pixels = lab_image_eq.reshape((-1, 3))

    # Apply k-means clustering
    k = 6  # You may need to adjust this based on your specific image
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(pixels)

    # Get cluster centers and labels
    centers = np.uint8(kmeans.cluster_centers_)
    labels = kmeans.labels_

    # Assign each pixel to the nearest cluster center
    segmented_image = centers[labels].reshape(lab_image_eq.shape)

    # Create a binary mask for the background
    background_mask = np.all(segmented_image == centers[0], axis=-1)

    # Apply the mask to the original image
    result = np.where(background_mask[:, :, None], 255, image)

    # Get relative path from the original image path
    relative_path = os.path.relpath(image_path, start=os.path.join(dataset_root, "train"))

    # Generate the output path based on the relative path
    output_path = os.path.join(output_root, relative_path)

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Save the processed image
    cv2.imwrite(output_path, cv2.cvtColor(result, cv2.COLOR_RGB2BGR))

# Specify the root directory of your dataset
dataset_root = r'D:\New_Fourth_Year\projectDesignGit\pestInfestationSeverityLevel\Sugarcane Leaf Disease Dataset Split'

# Specify the output root directory
output_root = r'D:\New_Fourth_Year\projectDesignGit\pestInfestationSeverityLevel\Sugarcane Leaf Disease Split No Background'

# Iterate through each image in the dataset
for folder_name in os.listdir(os.path.join(dataset_root, "train")):
    folder_path = os.path.join(dataset_root, "train", folder_name)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(folder_path, filename)
                process_image(image_path, output_root)

c:\Users\Jerico\anaconda3\envs\useGPU\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Jerico\anaconda3\envs\useGPU\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Jerico\anaconda3\envs\useGPU\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Jerico\anaconda3\envs\useGPU\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` wi

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 179, in where
KeyboardInterrupt: 
c:\Users\Jerico\anaconda3\envs\useGPU\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Jerico\anaconda3\envs\useGPU\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Jerico\anaconda3\envs\useGPU\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress t

In [1]:
import os
import concurrent.futures
import numpy as np
import cv2
from sklearn.cluster import KMeans

def process_image(image_path, output_root):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image to RGB format

    # Histogram Equalization
    lab_image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    l_channel_eq = cv2.equalizeHist(l_channel)
    lab_image_eq = cv2.merge([l_channel_eq, a_channel, b_channel])
    image_eq = cv2.cvtColor(lab_image_eq, cv2.COLOR_LAB2RGB)

    # Convert the equalized image to LAB
    lab_image_eq = cv2.cvtColor(image_eq, cv2.COLOR_RGB2LAB)

    # Reshape the image
    pixels = lab_image_eq.reshape((-1, 3))

    # Apply k-means clustering
    k = 6  # You may need to adjust this based on your specific image
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(pixels)

    # Get cluster centers and labels
    centers = np.uint8(kmeans.cluster_centers_)
    labels = kmeans.labels_

    # Assign each pixel to the nearest cluster center
    segmented_image = centers[labels].reshape(lab_image_eq.shape)

    # Create a binary mask for the background
    background_mask = np.all(segmented_image == centers[0], axis=-1)

    # Apply the mask to the original image
    result = np.where(background_mask[:, :, None], 255, image)

    # Get relative path from the original image path
    relative_path = os.path.relpath(image_path, start=os.path.join(dataset_root, "train"))

    # Generate the output path based on the relative path
    output_path = os.path.join(output_root, relative_path)

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Save the processed image
    cv2.imwrite(output_path, cv2.cvtColor(result, cv2.COLOR_RGB2BGR))

# Specify the root directory of your dataset
dataset_root = r'D:\New_Fourth_Year\projectDesignGit\pestInfestationSeverityLevel\Sugarcane Leaf Disease Dataset Split'

# Specify the output root directory
output_root = r'D:\New_Fourth_Year\projectDesignGit\pestInfestationSeverityLevel\Sugarcane Leaf Disease Split No Background'

# List to store image paths
image_paths = []

# Iterate through each image in the dataset and collect image paths
for folder_name in os.listdir(os.path.join(dataset_root, "train")):
    folder_path = os.path.join(dataset_root, "train", folder_name)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(folder_path, filename)
                image_paths.append(image_path)

# Function to process an image using process_image function
def process_image_wrapper(image_path):
    process_image(image_path, output_root)

# Set the number of threads (workers)
num_threads = 4  # Change this based on your system's capabilities
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Process each image concurrently
    executor.map(process_image_wrapper, image_paths)


c:\Users\Jerico\anaconda3\envs\useGPU\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Jerico\anaconda3\envs\useGPU\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


: 